# Serving Alibi-Detect models

Out of the box, `mlserver` supports the deployment and serving of `alibi-detect` models. In this example, we will cover how we can create a detector configuration to then serve it using `mlserver`.

## Reference Data and Configuration

The first step will be to fetch a reference data for an `alibi-detect` model. For that, we will use the [income Classifier example from the `alibi-detect` documentation](https://docs.seldon.io/projects/alibi-detect/en/latest/examples/cd_chi2ks_adult.html)

Install Alibi dependencies for dataset and detector creation

In [ ]:
!pip install alibi alibi_detect

In [3]:
import alibi
import matplotlib.pyplot as plt
import numpy as np

In [4]:
adult = alibi.datasets.fetch_adult()
X, y = adult.data, adult.target
feature_names = adult.feature_names
category_map = adult.category_map
X.shape, y.shape

((32561, 12), (32561,))

In [5]:
n_ref = 10000
n_test = 10000

X_ref, X_t0, X_t1 = X[:n_ref], X[n_ref:n_ref + n_test], X[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

((10000, 12), (10000, 12), (10000, 12))

In [6]:
categories_per_feature = {f: None for f in list(category_map.keys())}

In [21]:
cols = list(category_map.keys())
cat_names = [feature_names[_] for _ in list(category_map.keys())]
X_ref_cat, X_t0_cat = X_ref[:, cols], X_t0[:, cols]
X_ref_cat.shape, X_t0_cat.shape

((10000, 8), (10000, 8))

# TabularDrift

### Creating detector and saving configuration

In [22]:
from alibi_detect.cd import TabularDrift
cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature)

In [23]:
from alibi_detect.utils.saving import save_detector
filepath = "alibi-detector-artifacts/detector_data_tabular"
save_detector(cd, filepath)

### Detecting Drift locally

In [24]:
preds = cd.predict(X_t0,drift_type="feature")

labels = ['No!', 'Yes!']
print(f"Threshold {preds['data']['threshold']}")
for f in range(cd.n_features):
    fname = feature_names[f]
    is_drift = (preds['data']['p_val'][f] < preds['data']['threshold']).astype(int)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- Chi2 {stat_val:.3f} -- p-value {p_val:.3f}')

Threshold 0.05
Age -- Drift? No! -- Chi2 0.012 -- p-value 0.508
Workclass -- Drift? No! -- Chi2 8.487 -- p-value 0.387
Education -- Drift? No! -- Chi2 4.753 -- p-value 0.576
Marital Status -- Drift? No! -- Chi2 3.160 -- p-value 0.368
Occupation -- Drift? No! -- Chi2 8.194 -- p-value 0.415
Relationship -- Drift? No! -- Chi2 0.485 -- p-value 0.993
Race -- Drift? No! -- Chi2 0.587 -- p-value 0.965
Sex -- Drift? No! -- Chi2 0.217 -- p-value 0.641
Capital Gain -- Drift? No! -- Chi2 0.002 -- p-value 1.000
Capital Loss -- Drift? No! -- Chi2 0.002 -- p-value 1.000
Hours per week -- Drift? No! -- Chi2 0.012 -- p-value 0.508
Country -- Drift? No! -- Chi2 9.991 -- p-value 0.441


## Serving

Now that we have the reference data and other configuration parameters, the next step will be to serve it using `mlserver`. 
For that, we will need to create 2 configuration files: 

- `settings.json`: holds the configuration of our server (e.g. ports, log level, etc.).
- `model-settings.json`: holds the configuration of our model (e.g. input type, runtime to use, etc.).

### `settings.json`

In [11]:
%%writefile settings.json
{
    "debug": "true"
}

Overwriting settings.json


### `model-settings.json`

In [32]:
%%writefile model-settings.json
{
  "name": "income-classifier-cd",
  "implementation": "mlserver_alibi_detect.AlibiDetectRuntime",
  "parameters": {
    "uri": "./alibi-detector-artifacts/detector_data_tabular",
    "version": "v0.1.0"
  }
}

Overwriting model-settings.json


### Start serving our model

Now that we have our config in-place, we can start the server by running `mlserver start .`. This needs to either be ran from the same directory where our config files are or pointing to the folder where they are.

```shell
mlserver start .
```

Since this command will start the server and block the terminal, waiting for requests, this will need to be ran in the background on a separate terminal.

### Send test inference request

We now have our model being served by `mlserver`.
To make sure that everything is working as expected, let's send a request from our test set.

For that, we can use the Python types that `mlserver` provides out of box, or we can build our request manually.

### Detecting Drift via MLServer

In [15]:
import requests

inference_request = {
    "inputs": [
        {
            "name": "predict",
            "shape": X_t0.shape,
            "datatype": "FP32",
            "data": X_t0.tolist(),
        }
    ],
    "parameters":{
        "predict_kwargs":{
            "drift_type":"feature"
        }
    }
}

endpoint = "http://localhost:8080/v2/models/income-classifier-cd/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

In [16]:
import json
response_dict = json.loads(response.text)
print(response_dict,"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = response_dict['outputs'][0]['data'][f]
    print(f'{fname} -- Drift? {labels[is_drift]}')

{'model_name': 'income-classifier-cd', 'model_version': 'v0.1.0', 'id': '72789455-2633-492f-99c8-b47fc5978149', 'parameters': {'content_type': None, 'headers': None, 'predict_kwargs': {}, 'detector_type': 'offline', 'name': 'TabularDrift', 'data_type': None, 'version': '0.8.2dev', 'config_file': '/home/sachin/projects/seldon/MLServer/docs/examples/alibi-detect/alibi-detector-artifacts/detector_data_tabular/config.toml'}, 'outputs': [{'name': 'is_drift', 'shape': [1, 12], 'datatype': 'INT64', 'parameters': None, 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'name': 'distance', 'shape': [1, 12], 'datatype': 'FP32', 'parameters': None, 'data': [0.011599999852478504, 8.486513137817383, 4.752931594848633, 3.1599440574645996, 8.194136619567871, 0.4845852553844452, 0.5865231156349182, 0.21689055860042572, 0.002400000113993883, 0.0015999999595806003, 0.011599999852478504, 9.991032600402832]}, {'name': 'p_val', 'shape': [1, 12], 'datatype': 'FP32', 'parameters': None, 'data': [0.5078648328781

# ChiSquareDrift

### Creating detector and saving configuration

In [42]:
from alibi_detect.cd import ChiSquareDrift
cd = ChiSquareDrift(X_ref_cat, p_val=.05)

In [43]:
from alibi_detect.utils.saving import save_detector
filepath = "alibi-detector-artifacts/detector_data_cat"
save_detector(cd, filepath)

### Detecting Drift locally

In [44]:
preds = cd.predict(X_t0_cat,drift_type="feature")

labels = ['No!', 'Yes!']
print(f"Threshold {preds['data']['threshold']}")
for f in range(cd.n_features):
    fname = cat_names[f]
    is_drift = (preds['data']['p_val'][f] < preds['data']['threshold']).astype(int)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- Chi2 {stat_val:.3f} -- p-value {p_val:.3f}')

Threshold 0.05
Workclass -- Drift? No! -- Chi2 8.487 -- p-value 0.387
Education -- Drift? No! -- Chi2 4.753 -- p-value 0.576
Marital Status -- Drift? No! -- Chi2 3.160 -- p-value 0.368
Occupation -- Drift? No! -- Chi2 8.194 -- p-value 0.415
Relationship -- Drift? No! -- Chi2 0.485 -- p-value 0.993
Race -- Drift? No! -- Chi2 0.587 -- p-value 0.965
Sex -- Drift? No! -- Chi2 0.217 -- p-value 0.641
Country -- Drift? No! -- Chi2 9.991 -- p-value 0.441


## Serving

Now that we have the reference data and other configuration parameters, the next step will be to serve it using `mlserver`. 
For that, we will need to create 2 configuration files: 

- `settings.json`: holds the configuration of our server (e.g. ports, log level, etc.).
- `model-settings.json`: holds the configuration of our model (e.g. input type, runtime to use, etc.).

### `settings.json`

In [45]:
%%writefile settings.json
{
    "debug": "true"
}

Overwriting settings.json


### `model-settings.json`

In [52]:
%%writefile model-settings.json
{
  "name": "income-classifier-cd",
  "implementation": "mlserver_alibi_detect.AlibiDetectRuntime",
  "parameters": {
    "uri": "./alibi-detector-artifacts/detector_data_cat",
    "version": "v0.1.0"
  }
}

Overwriting model-settings.json


### Start serving our model

Now that we have our config in-place, we can start the server by running `mlserver start .`. This needs to either be ran from the same directory where our config files are or pointing to the folder where they are.

```shell
mlserver start .
```

Since this command will start the server and block the terminal, waiting for requests, this will need to be ran in the background on a separate terminal.

### Send test inference request

We now have our model being served by `mlserver`.
To make sure that everything is working as expected, let's send a request from our test set.

For that, we can use the Python types that `mlserver` provides out of box, or we can build our request manually.

In [53]:
import requests

inference_request = {    
    "inputs": [
        {
            "name": "predict",
            "shape": X_t0_cat.shape,
            "datatype": "FP32",
            "data": X_t0_cat.tolist(),
        }
    ],
    "parameters":{
        "predict_kwargs":{
            "drift_type":"feature"
        }
    }
}

endpoint = "http://localhost:8080/v2/models/income-classifier-cd/versions/v0.1.0/infer"
response = requests.post(endpoint, json=inference_request)

In [54]:
import json
response_dict = json.loads(response.text)
print(response_dict,"\n")

labels = ['No!', 'Yes!']
for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = response_dict['outputs'][0]['data'][f]
    print(f'{fname} -- Drift? {labels[is_drift]}')

{'model_name': 'income-classifier-cd', 'model_version': 'v0.1.0', 'id': '4daca232-95b3-44fa-bd15-c286cdf39d5c', 'parameters': {'content_type': None, 'headers': None, 'predict_kwargs': {}, 'detector_type': 'offline', 'config_file': '/home/sachin/projects/seldon/MLServer/docs/examples/alibi-detect/alibi-detector-artifacts/detector_data_cat/config.toml', 'version': '0.8.2dev', 'name': 'ChiSquareDrift', 'data_type': None}, 'outputs': [{'name': 'is_drift', 'shape': [1, 8], 'datatype': 'INT64', 'parameters': None, 'data': [0, 0, 0, 0, 0, 0, 0, 0]}, {'name': 'distance', 'shape': [1, 8], 'datatype': 'FP32', 'parameters': None, 'data': [8.486513137817383, 4.752931594848633, 3.1599440574645996, 8.194136619567871, 0.4845852553844452, 0.5865231156349182, 0.21689055860042572, 9.991032600402832]}, {'name': 'p_val', 'shape': [1, 8], 'datatype': 'FP32', 'parameters': None, 'data': [0.3874434530735016, 0.5758693218231201, 0.3676162362098694, 0.4147398769855499, 0.992676854133606, 0.9645494222640991, 0.